In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize, minimize_scalar
import sys
sys.path.append("..")
from src.utils import contact_matrix

Computing the exact likelihood for the CRKP transmission model.

$$
    \log \mathcal{L}(\beta ; \mathbf{X}) = \sum_{t=2}^T \sum_{i=1}^N [(1 - X_{t-1}^{(i)})W_t^{(i)}W_{t-1}^{(i)}] \cdot [X_t^{(i)} \log (1 - e^{- \lambda_i(t)}) + (1 - X_t^{(i)}) \log e^{- \lambda_i(t)})]
$$

In [ ]:
path = "/Volumes/umms-esnitkin/Project_KPC_LTACH/Analysis/LTACH_transmission_modeling/preprocessed/resampled"

In [ ]:
X = pd.read_csv(f"{path}/infections.csv", index_col=0).values
W = pd.read_csv(f"{path}/facility_trace.csv", index_col=0).values
V = pd.read_csv(f"{path}/screening.csv", index_col=0).values
F = pd.read_csv(f"{path}/floor_trace.csv", index_col=0).values
R = pd.read_csv(f"{path}/room_trace.csv", index_col=0).values
cap = np.array([129., 28., 38., 35., 27., 17., 2])

In [ ]:
def nll(beta, X, W, V, F, R, het=False):
    beta_0 = beta[0] if het else beta
    N, T = W.shape
    # X = np.empty((N, T))
    # load screen data for first day
    room_count = np.empty(T)
    ans = 0
    for t in range(1, T):
        x = X[:, t-1]
        w = W[:, t-1]
        f = F[:, t-1]
        r = R[:, t-1]
        # who was infected at the last timestep?
        Ia = x[w > 0]
        hazard = Ia.sum() * beta_0 * np.ones(N) / cap[0]
        fa = f[w > 0]
        fC = contact_matrix(fa)
        # guarantee that there are no infecteds who aren't present
        # how many infected floormates?
        if het:
            hazard[w > 0] += (fC * Ia).sum(1) * beta[fa] / cap[fa]
        ra = r[w > 0]
        rC = contact_matrix(ra)
        infected_roommates = (rC * Ia).sum(1)
        if het:
            hazard[w > 0] += infected_roommates * beta[-1] / cap[-1]
        p = 1 - np.exp(-hazard)
    
        xt = X[:, t]
    
        valid = W[:, t] * w * (1 - xt) # present for at least week & susceptible

        ans += np.nansum(valid * ((xt * np.log(p)) + (1 - xt) * np.log(1 - p)))

        print(hazard)
        print(p)

        break

    return ans

### homogeneous model

In [ ]:
def f(beta):
    return - nll(beta, X, W, V, F, R)

In [ ]:
res = minimize_scalar(f, bounds = (0.01, 1), method="bounded", options={"disp": True, "maxiter": 18})

In [ ]:
f(0.13)